#### Train different classifiers on still image datasets and visualize their outputs/performance

PARAMETERS
* detection confidence  -  check from realtime detection
* tracking confidence  -  check from realtime detection
* presence confidence  -  check from realtime detection
* CLASSIFIER: random forest - n_estimators

class balancing or classifier does not do much - seems to be a matter of the
kyepoint detection

In [89]:
import numpy as np
import mediapipe as mp
import pickle
from prepare_data import parse_image_folder
from extract_features import *
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import cv2

seed = 333

In [68]:
# from the Gemini API

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [2]:
# still image dataset path
directory = "/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/image_data"

# get standardized images and labels
X, labels, df = parse_image_folder(directory)

print('Number of images:', len(X))

Number of images: 2754


In [180]:
# extract landmarks: MEDIAPIPE 
min_pose_detection_confidence = 0.5  # biggest effect
min_pose_presence_confidence = 0.5
min_tracking_confidence = 0.8

mp_model_path = "../pretrained_models/pose_landmarker_full.task"
detector = mediapipe_detector(mp_model_path,
                              min_pose_detection_confidence=min_pose_detection_confidence,
                              min_pose_presence_confidence=min_pose_presence_confidence,
                              min_tracking_confidence=min_tracking_confidence,)

x_data = np.zeros((len(X), MP_N_LANDMARKS * 4))  # For pose landmarks
y_data = np.zeros(labels.shape)  # For labels

c = 0

# run inference on every image
for i in range(len(X)):
    landmarks = mediapipe_detect(detector, X[i])
    # only include images with detected landmarks
    if landmarks is not None:
        # with detected poses
        if len(landmarks.pose_landmarks) != 0:
            x_data[i], y_data[i] = mediapipe_format_landmark(landmarks, labels[i])
            
            # draw one example detection
            if c < 100:
                annotated_image = draw_landmarks_on_image(X[i], landmarks)
                cv2.imwrite(f'visualize_landmarks/{i}.jpg', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))     
                c += 1                        

# drop images with no landmarks detected
non_zero_mask = np.any(x_data != 0, axis=1)
x_data = x_data[non_zero_mask]
y_data = y_data[y_data != 0]

# train-test split
X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, shuffle=True, random_state=seed
)

print('Number of training samples:', len(X_train))

I0000 00:00:1733701873.223434 50116083 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Number of training samples: 1749


In [181]:
# save the landmark training datasets

with open(f'landmark_datasets/landmark_train_{min_pose_detection_confidence}_{min_pose_presence_confidence}_{min_tracking_confidence}.pkl', 'wb') as f:
    pickle.dump([X_train, y_train], f)
    
with open(f'landmark_datasets/landmark_test_{min_pose_detection_confidence}_{min_pose_presence_confidence}_{min_tracking_confidence}.pkl', 'wb') as f:
    pickle.dump([X_test, y_test], f)

In [182]:
# class balancing - not necessarily better
smote_k_neighbors = 3  
smote = SMOTE(random_state=seed, k_neighbors=smote_k_neighbors)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# save class distributions before and after rebalancing
y_before_rebalance = pd.Series(y_train).value_counts()
y_after_rebalance = pd.Series(y_resampled).value_counts()
with open(f'rebalancing_analysis/SMOTE_{smote_k_neighbors}.pkl', 'wb') as f:
    pickle.dump((y_before_rebalance, y_after_rebalance), f)
    
print('Number of training samples after SMOTE:', len(X_resampled))

Number of training samples after SMOTE: 3478


In [204]:
# #train classifier 
# forest_n_estimators = 300  # upper bound here
# forest = RandomForestClassifier(n_estimators=forest_n_estimators, random_state=seed, verbose=1)
# forest.fit(X_resampled, y_resampled)

# # save the model
# with open(f'../trained_classifiers/random_forest.pkl', 'wb') as f:
#     pickle.dump(forest, f)

# # train a very simple neural network to get the classification
# from sklearn.neural_network import MLPClassifier
# nn = MLPClassifier(random_state=seed, 
#                    max_iter=2000, 
#                    alpha =0.001,
#                    learning_rate_init = 0.001,
#                    hidden_layer_sizes=(100,50,100,50),)
# nn.fit(X_resampled, y_resampled)

# # # save the model
# # with open(f'../trained_classifiers/neural_network.pkl', 'wb') as f:
# #     pickle.dump(nn, f)

# # train another type of classifier
# from sklearn.svm import SVC
# svm = SVC(random_state=seed)
# svm.fit(X_resampled, y_resampled)

# # save the model
# with open(f'../trained_classifiers/svm.pkl', 'wb') as f:
#     pickle.dump(svm, f)

# train a classifier
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=seed)
gbc.fit(X_resampled, y_resampled)

# save the model
with open(f'../trained_classifiers/gradient_boosting.pkl', 'wb') as f:
    pickle.dump(gbc, f)

In [203]:
# evaluate model

# NOTE: NEWneural_network worked w/o being trained on the last??

y_pred = gbc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Mediapipe Model Accuracy (with SMOTE): {accuracy}")


Mediapipe Model Accuracy (with SMOTE): 0.684931506849315
Mediapipe Model Accuracy (with SMOTE): 0.3150684931506849


/Users/alejandraduran/opt/anaconda3/envs/cos429/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/alejandraduran/opt/anaconda3/envs/cos429/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
